In [1]:
import numpy as np
import keras
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Conv2D
from keras.layers import MaxPooling2D, UpSampling2D, Flatten, Reshape
from keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
import pandas as pd
import os
from PIL import Image


h = 256
w = 256
ch = 1

trainlist = pd.read_table("/Users/akipin/opt/AutoEncoderPelvis/normal.txt", header=0)
train_data = np.zeros((len(trainlist["filenames"]),w,h,ch))

for i in range(0, len(trainlist)):
    tmpfile = trainlist["filenames"][i]
    train_data[i] = np.array(img_to_array(load_img(tmpfile, grayscale=True, target_size=(h, w))))/255.0

Using TensorFlow backend.
/Users/akipin/opt/anaconda3/envs/dls-2/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/akipin/opt/anaconda3/envs/dls-2/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/akipin/opt/anaconda3/envs/dls-2/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/a

In [14]:
input_img = Input(shape=(h, w, ch))
encoding_dim = 6

#Encocder
x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Flatten()(x)
encoded = Dense(encoding_dim, activation='relu')(x)

#Decoder
x = Dense(4*4*8, activation='relu')(encoded)
x = Reshape((4, 4, 8))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
encoder = Model(input_img, encoded)

autoencoder.compile(optimizer='Adam', loss='mse')

In [15]:
autoencoder.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 256, 256, 1)       0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 256, 256, 64)      640       
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 128, 128, 128)     73856     
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 64, 64, 128)       147584    
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 32, 32, 128)       0   

In [16]:
encoder.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 256, 256, 1)       0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 256, 256, 64)      640       
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 128, 128, 128)     73856     
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 64, 64, 128)       147584    
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 32, 32, 128)       0   

In [ ]:
autoencoder-fit(
    train_data,
    train_data,
    epochs=3,
    batch_size=128,
    shuffle=True
)